# Results Tables

The success of an assembly is evaluated by the computation of metrics in two defined ways: globally through statistics inherent to the complete set of sequences that were assembled, and relative to the replicons present in the sample. 

The computation of the global metrics is done through custom python code from the complete set of sequences assembled. 

The following metrics are computed for the complete and filtered set of assembled sequences, restricted to contigs of length above a specified minimum size: 

- **Contig sizes**
    - **Contigs:** The total number of contigs in the assembly;
    - **Basepairs:** The total number of bases in the assembly;
    - **Maximum sequence length:** The length of the largest contig in the assembly.
    - **Number of ‘N’s:** Number of uncalled bases (N's) 
- **Contiguity**
    - **Nx (where 0  < x  ⩽ 100):** Length for which the collection of all assembled sequences of that length or longer in an assembly covers at least a given percentage of the total length of the assembly
- **Misassembly**
    - **Misassemblies** - Number of aligned contigs that contain a misassembly event

After filtering, the sequences are mapped with the reference replicons and the metrics are computed through custom python code. The metrics are calculated, filtered for a minimum length, for each replicon in the file provided input references. 

- **Contig sizes**
    - **Contigs:** The total number of contigs in the assembly;
    - **Basepairs:** The total number of bases in the assembly;
    - **Number of ‘N’s:** Number of uncalled bases (N's) 
- **COMPASS**
    - **(Breadth of) Coverage:** Ratio of covered sequence on the reference by aligned contigs;
    - **Multiplicity:** Ratio of the length of alignable assembled sequence to covered sequence on the reference;
    - **Validity:** Ratio of the length of the alignable assembled sequence to total basepairs in the aligned contigs;
    - **Parsimony:** Cost of the assembly (multiplicity over validity);
- **Contiguity**
    - **Contiguity:** longest single alignment between the assembly and the reference, relative to the reference length;
    - **NAx (where 0  < x  ⩽ 100):** Length for which the collection of aligned assembled sequences of that length or longer in an assembly covers at least a given percentage of the total length of the reference replicon;
    - **NGx (where 0  < x  ⩽ 100):** Length for which the collection of aligned contigs of that length or longer covers at least a given percentage of the sequence of the reference.
    - **Lx (where 0  < x  ⩽ 100):** Minimal number of contigs that cover x % of the sequence of the reference;
- **Identity**
    - **Identity:** Ratio of identical basepairs in all aligned contigs to the reference;
    - **Lowest identity:** Identity of the lowest scoring contig to the reference.
    - **PLS**: Phred-like score per contig, per assembler.
- **Misassembly**
    - **Misassemblies** - Number of aligned contigs that contain a misassembly event


## Imports

In [1]:
import sys
from plotly.offline import plot
import glob
import fnmatch
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import json
import pandas as pd
from itertools import groupby
import csv
import numpy as np

## Global variables

In [2]:
METRICS_COLUMNS_GLOBAL = ['run','sample','assembler', 'contigs', 'basepairs', 'max_contig', 'Ns', 'n50', 
                          'misassemblies', 'mapped_reads', 'filtered_contigs', 'filtered_basepairs',
                          'filtered_n50', 'filtered_Ns', 'type', 'algorythm']
METRICS_COLUMNS_REFERENCE = ['run','sample','assembler', 'reference', 'contiguity', 'breadth_of_coverage', 'multiplicity', 'validity', 
                             'parsimony', 'identity', 'lowest_identity', 'L90', 'contigs', 'NA50', 'NG50', 'basepairs', 'Ns',
                             'misassemblies', 'type', 'algorythm']

ASSEMBLER_PROCESS_LIST = ["BCALM2", "GATBMINIAPIPELINE", "MINIA", "MEGAHIT", "METASPADES", "UNICYCLER", "SPADES",
                          "SKESA", "VELVETOPTIMIZER", "IDBA"]
PROCESS_TO_NAME = {"BCALM2": "BCALM2", 
                   "GATBMINIAPIPELINE": "GATBMiniaPipeline",
                   "MINIA": "MINIA",
                   "MEGAHIT": "MEGAHIT", 
                   "METASPADES": "metaSPAdes", 
                   "UNICYCLER": "Unicycler", 
                   "SPADES": "SPAdes",
                    "SKESA": "SKESA",
                   "VELVETOPTIMIZER": "VelvetOptimizer",
                   "IDBA": "IDBA-UD"}

log_distributed = ['LHS', 'LNN', 'ERR2935805']
even_distribution = ['EMS', 'LNN', 'ERR2984773']

genomic_assemblers = ['BCALM2', 'MINIA', 'SKESA', 'SPAdes', 'Unicycler', 'VelvetOptimizer']
metagenomic_assemblers = ['GATBMiniaPipeline', 'IDBA-UD', 'MEGAHIT', 'metaSPAdes']
single_kmer = ['BCALM2', 'MINIA']
multiple_kmer = ['SKESA', 'SPAdes', 'Unicycler', 'VelvetOptimizer', 'GATBMiniaPipeline', 
                 'IDBA-UD', 'MEGAHIT', 'metaSPAdes']

## Global metrics

### Load data

In [3]:
report_glob = glob.glob('../Results/*/report/pipeline_report_tables.json')
global_pipeline_metrics_df = pd.DataFrame(columns=METRICS_COLUMNS_GLOBAL)

for pipeline_report_file in report_glob:
    report_file_name = pipeline_report_file.split('/')[-1]
    stats_run = pipeline_report_file.split('/')[-3]
    print('Processing {0} data from {1}...'.format(report_file_name, stats_run))
    
    with open(pipeline_report_file) as _fh:
        json_report = json.load(_fh)
        for sample in json_report.keys():
            for line in json_report[sample]['GlobalTable']:
                assembler = line['assembler']
                global_pipeline_metrics_df = global_pipeline_metrics_df.append({'run': stats_run,
                                                                                'sample': sample,
                                                                                'assembler': line['assembler'],
                                                                                'contigs': int(line['original']['contigs']),
                                                                                'basepairs': int(line['original']['basepairs']),
                                                                                'max_contig': int(line['original']['max_contig_size']),
                                                                                'n50': int(line['original']['N50']),
                                                                                'mapped_reads': line['original']['mapped_reads'],
                                                                                'Ns': int(line['original']['Ns']),
                                                                                'misassemblies': line['filtered']['misassembled_contigs'],
                                                                                'filtered_contigs': line['filtered']['contigs'],
                                                                                'filtered_basepairs': line['filtered']['basepairs'],
                                                                                'filtered_n50': line['filtered']['N50'],
                                                                                'filtered_Ns': line['filtered']['Ns']
                                                                               },
                                                                               ignore_index=True)
global_pipeline_metrics_df['distribution'] = np.where(global_pipeline_metrics_df['sample'].isin(log_distributed), 'Log', 'Even')
global_pipeline_metrics_df['type'] = np.where(global_pipeline_metrics_df['assembler'].isin(genomic_assemblers), 'Genomic', 'Metagenomic')
global_pipeline_metrics_df['algorythm'] = np.where(global_pipeline_metrics_df['assembler'].isin(single_kmer), 'Single k-mer De Bruijn graph', 'Multiple k-mer De Bruijn graph')
global_pipeline_metrics_df[['contigs','basepairs','max_contig', 'Ns','n50','misassemblies']] = global_pipeline_metrics_df[['contigs','basepairs','max_contig', 'Ns','n50','misassemblies']].apply(pd.to_numeric)

Processing pipeline_report_tables.json data from run3...
Processing pipeline_report_tables.json data from run2...


### Preview data

In [4]:
global_pipeline_metrics_df.dtypes

run                    object
sample                 object
assembler              object
contigs                 int64
basepairs               int64
max_contig              int64
Ns                      int64
n50                     int64
misassemblies           int64
mapped_reads          float64
filtered_contigs       object
filtered_basepairs     object
filtered_n50           object
filtered_Ns            object
type                   object
algorythm              object
distribution           object
dtype: object

In [5]:
global_pipeline_metrics_df

,run,sample,assembler,contigs,basepairs,max_contig,Ns,n50,misassemblies,mapped_reads,filtered_contigs,filtered_basepairs,filtered_n50,filtered_Ns,type,algorythm,distribution
0,run3,ENN,BCALM2,25258,31040696,161812,0,17766,0,96.832836,3542,28188554,21029,0,Genomic,Single k-mer De Bruijn graph,Even
1,run3,ENN,GATBMiniaPipeline,1008,30445418,632215,0,106963,0,99.480265,727,30282639,106963,0,Metagenomic,Multiple k-mer De Bruijn graph,Even
2,run3,ENN,IDBA-UD,1825,30390287,468605,0,61778,12,99.414283,1279,30136999,62331,0,Metagenomic,Multiple k-mer De Bruijn graph,Even
3,run3,ENN,MEGAHIT,810,30541296,632274,0,170162,9,99.731417,517,30415305,171359,0,Metagenomic,Multiple k-mer De Bruijn graph,Even
4,run3,ENN,metaSPAdes,633,30448137,1036942,0,234919,1,99.596146,350,30376419,234919,0,Metagenomic,Multiple k-mer De Bruijn graph,Even
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,run2,LHS,MINIA,24652,15334650,163543,0,7727,0,91.099517,2171,12920930,10582,0,Genomic,Single k-mer De Bruijn graph,Log
116,run2,LHS,SKESA,36,2957060,632013,0,351027,0,92.228441,21,2950393,351027,0,Genomic,Multiple k-mer De Bruijn graph,Log
117,run2,LHS,SPAdes,31225,16224766,263645,0,46478,1,95.503607,1526,13088051,69361,0,Genomic,Multiple k-mer De Bruijn graph,Log
118,run2,LHS,Unicycler,276,13345058,632062,0,187479,3,96.948570,247,13333597,187479,0,Genomic,Multiple k-mer De Bruijn graph,Log


### Save data

In [6]:
global_pipeline_metrics_df.to_csv("Tables/Results/Global metrics.csv")

### Global statistics per assembler type

In [7]:
stats_per_assembler_type = pd.DataFrame()
for assembly_type in global_pipeline_metrics_df.type.unique():
    row={'Assembler Type': assembly_type}
    lala=global_pipeline_metrics_df[global_pipeline_metrics_df.type == assembly_type].describe()
    for column in lala.columns:
        mean = lala.loc['mean',column]
        std = lala.loc['std',column]
        row[column] = "{}(±{})".format(round(mean, 2), round(std, 2))
    stats_per_assembler_type=stats_per_assembler_type.append(row, ignore_index=True)
stats_per_assembler_type = stats_per_assembler_type.set_index('Assembler Type')
stats_per_assembler_type.to_csv("Tables/Results/Global metrics per assembler type.csv")

In [8]:
stats_per_assembler_type

,Ns,basepairs,contigs,mapped_reads,max_contig,misassemblies,n50
Assembler Type,,,,,,,
Genomic,632.49(±2696.83),45049830.26(±77416335.87),584820.46(±1927506.27),74.63(±36.42),407391.93(±369952.0),18.17(±94.55),75438.93(±96288.64)
Metagenomic,0.0(±0.0),25547301.35(±10011582.38),8085.02(±13528.83),94.46(±6.1),706300.71(±216738.75),4.06(±4.6),116760.52(±64826.32)


### Global stats per assembly algorithm

In [9]:
stats_per_assembler_algo = pd.DataFrame()
for assembly_algo in global_pipeline_metrics_df.algorythm.unique():
    row={'Assembler Algorithm': assembly_algo}
    lala=global_pipeline_metrics_df[global_pipeline_metrics_df.algorythm == assembly_algo].describe()
    for column in lala.columns:
        mean = lala.loc['mean',column]
        std = lala.loc['std',column]
        row[column] = "{}(±{})".format(round(mean, 2), round(std, 2))
    stats_per_assembler_algo=stats_per_assembler_algo.append(row, ignore_index=True)
stats_per_assembler_algo = stats_per_assembler_algo.set_index('Assembler Algorithm')
stats_per_assembler_algo.to_csv("Tables/Results/Global metrics per assembler algorithm.csv")

In [10]:
stats_per_assembler_algo

,Ns,basepairs,contigs,mapped_reads,max_contig,misassemblies,n50
Assembler Algorithm,,,,,,,
Single k-mer De Bruijn graph,0.0(±0.0),91615808.0(±121102826.02),1691066.88(±3086055.78),68.91(±38.55),108171.83(±74665.4),0.33(±0.76),10206.33(±9056.65)
Multiple k-mer De Bruijn graph,474.36(±2347.62),23657071.38(±12649027.02),19891.14(±86129.94),85.97(±26.66),631651.34(±309648.34),15.57(±81.88),112407.88(±85994.72)


### Global stats per assembler

In [11]:
stats_per_assembler = pd.DataFrame()
for assembler in global_pipeline_metrics_df.assembler.unique():
    row={'Assembler': assembler, 
         'Type':global_pipeline_metrics_df[global_pipeline_metrics_df.assembler == assembler].type.unique()[0],
         'Algorithm': global_pipeline_metrics_df[global_pipeline_metrics_df.assembler == assembler].algorythm.unique()[0]}
    lala=global_pipeline_metrics_df[global_pipeline_metrics_df.assembler == assembler].describe()
    for column in lala.columns:
        mean = lala.loc['mean',column]
        std = lala.loc['std',column]
        row[column] = "{}(±{})".format(round(mean, 2), round(std, 2))
    stats_per_assembler=stats_per_assembler.append(row, ignore_index=True)
stats_per_assembler = stats_per_assembler.set_index('Assembler')
stats_per_assembler.to_csv("Tables/Results/Global metrics per assembler.csv")

In [12]:
stats_per_assembler

,Algorithm,Ns,Type,basepairs,contigs,mapped_reads,max_contig,misassemblies,n50
Assembler,,,,,,,,,
BCALM2,Single k-mer De Bruijn graph,0.0(±0.0),Genomic,157403460.17(±145361404.71),3352423.83(±3727041.83),46.33(±44.2),58728.17(±77129.19),0.0(±0.0),6220.67(±8873.1)
GATBMiniaPipeline,Multiple k-mer De Bruijn graph,0.0(±0.0),Metagenomic,23074937.42(±8661768.88),2629.42(±2589.73),94.65(±6.52),640091.33(±68954.44),1.92(±2.07),102424.0(±26843.99)
IDBA-UD,Multiple k-mer De Bruijn graph,0.0(±0.0),Metagenomic,25378779.83(±9698785.87),6832.17(±7856.47),93.22(±6.21),515095.83(±121784.53),6.17(±4.73),67858.42(±30786.39)
MEGAHIT,Multiple k-mer De Bruijn graph,0.0(±0.0),Metagenomic,25840461.5(±10055256.9),6528.5(±8264.11),94.98(±6.19),847862.5(±228686.73),6.33(±6.49),147830.83(±47776.33)
metaSPAdes,Multiple k-mer De Bruijn graph,0.0(±0.0),Metagenomic,27895026.67(±12052231.75),16350.0(±23107.65),94.97(±6.09),822153.17(±218316.11),1.83(±0.94),148928.83(±94815.17)
MINIA,Single k-mer De Bruijn graph,0.0(±0.0),Genomic,25828155.83(±9600619.33),29709.92(±19566.17),91.49(±6.43),157615.5(±19336.61),0.67(±0.98),14192.0(±7623.07)
SKESA,Multiple k-mer De Bruijn graph,0.0(±0.0),Genomic,18472455.67(±12490360.65),3671.5(±4090.4),90.6(±8.18),390219.83(±263956.92),3.33(±4.46),101495.67(±123485.57)
SPAdes,Multiple k-mer De Bruijn graph,0.0(±0.0),Genomic,28138604.33(±10873664.65),19436.5(±17375.62),94.21(±6.49),697457.83(±218426.88),98.0(±221.9),107840.17(±85499.81)
Unicycler,Multiple k-mer De Bruijn graph,0.0(±0.0),Genomic,21030768.0(±9911805.52),531.33(±330.3),94.07(±6.84),857592.0(±234205.23),1.33(±1.44),212762.33(±22226.12)


### Global stats per distribution

In [13]:
for distribution in global_pipeline_metrics_df.distribution.unique():
    stats_per_distribution = pd.DataFrame()
    for assembler in global_pipeline_metrics_df.assembler.unique():
        row={'Assembler': assembler, 
             'Type':global_pipeline_metrics_df[(global_pipeline_metrics_df.assembler == assembler) & (global_pipeline_metrics_df.distribution == distribution)].type.unique()[0],
             'Algorithm': global_pipeline_metrics_df[global_pipeline_metrics_df.assembler == assembler].algorythm.unique()[0]}
        lala=global_pipeline_metrics_df[(global_pipeline_metrics_df.assembler == assembler) & (global_pipeline_metrics_df.distribution == distribution)].describe()
        for column in lala.columns:
            mean = lala.loc['mean',column]
            std = lala.loc['std',column]
            row[column] = "{}(±{})".format(round(mean, 2), round(std, 2))
        stats_per_distribution=stats_per_distribution.append(row, ignore_index=True)
    stats_per_distribution = stats_per_distribution.set_index('Assembler')
    stats_per_distribution.to_csv("Tables/Results/{} - Global metrics per assembler.csv".format(distribution))
    print(distribution)
    display(stats_per_distribution)

Even


,Algorithm,Ns,Type,basepairs,contigs,mapped_reads,max_contig,misassemblies,n50
Assembler,,,,,,,,,
BCALM2,Single k-mer De Bruijn graph,0.0(±0.0),Genomic,81485532.0(±69082486.61),1286040.67(±1726341.64),57.29(±40.35),61507.0(±78246.58),0.0(±0.0),6186.0(±8975.2)
GATBMiniaPipeline,Multiple k-mer De Bruijn graph,0.0(±0.0),Metagenomic,31254576.33(±1205888.55),2439.5(±2292.43),93.87(±8.31),677707.0(±70298.6),2.67(±2.73),110537.33(±9542.15)
IDBA-UD,Multiple k-mer De Bruijn graph,0.0(±0.0),Metagenomic,33523229.67(±4845847.19),7943.0(±9486.87),90.89(±7.11),413513.0(±85348.16),9.67(±4.41),49048.33(±21762.56)
MEGAHIT,Multiple k-mer De Bruijn graph,0.0(±0.0),Metagenomic,34099350.67(±5474162.25),7463.33(±10186.7),94.35(±7.85),868834.0(±268046.42),12.0(±3.9),143276.33(±42581.65)
metaSPAdes,Multiple k-mer De Bruijn graph,0.0(±0.0),Metagenomic,36624891.67(±9555633.92),21002.33(±31475.51),94.28(±7.71),847661.67(±290003.13),2.0(±0.89),158678.0(±91558.71)
MINIA,Single k-mer De Bruijn graph,0.0(±0.0),Genomic,34158353.67(±5443602.16),33771.5(±26508.92),90.39(±7.82),147189.67(±22652.82),0.0(±0.0),14485.0(±5722.72)
SKESA,Multiple k-mer De Bruijn graph,0.0(±0.0),Genomic,30285330.33(±115680.54),1219.0(±199.11),92.53(±8.74),540293.0(±83135.61),2.67(±2.07),85120.33(±16926.29)
SPAdes,Multiple k-mer De Bruijn graph,0.0(±0.0),Genomic,35828349.33(±8294764.4),13949.33(±20031.01),94.27(±7.78),813593.33(±52845.57),4.33(±4.41),158629.67(±83121.53)
Unicycler,Multiple k-mer De Bruijn graph,0.0(±0.0),Genomic,30462753.0(±99796.14),833.67(±138.96),93.41(±8.6),1011262.0(±227172.01),1.67(±1.37),215168.67(±26920.81)


Log


,Algorithm,Ns,Type,basepairs,contigs,mapped_reads,max_contig,misassemblies,n50
Assembler,,,,,,,,,
BCALM2,Single k-mer De Bruijn graph,0.0(±0.0),Genomic,233321388.33(±166976037.91),5418807.0(±4163140.62),35.37(±48.8),55949.33(±83345.94),0.0(±0.0),6255.33(±9625.66)
GATBMiniaPipeline,Multiple k-mer De Bruijn graph,0.0(±0.0),Metagenomic,14895298.5(±1740108.73),2819.33(±3068.06),95.43(±4.81),602475.67(±46071.98),1.17(±0.75),94310.67(±36555.23)
IDBA-UD,Multiple k-mer De Bruijn graph,0.0(±0.0),Metagenomic,17234330.0(±4926154.48),5721.33(±6544.52),95.56(±4.6),616678.67(±24065.69),2.67(±0.52),86668.5(±27611.4)
MEGAHIT,Multiple k-mer De Bruijn graph,0.0(±0.0),Metagenomic,17581572.33(±5363708.41),5593.67(±6662.12),95.61(±4.66),826891.0(±205305.52),0.67(±0.52),152385.33(±56202.22)
metaSPAdes,Multiple k-mer De Bruijn graph,0.0(±0.0),Metagenomic,19165161.67(±6734121.74),11697.67(±11491.98),95.66(±4.59),796644.67(±138537.99),1.67(±1.03),139179.67(±105672.65)
MINIA,Single k-mer De Bruijn graph,0.0(±0.0),Genomic,17497958.0(±2569841.72),25648.33(±9996.14),92.59(±5.2),168041.33(±6968.79),1.33(±1.03),13899.0(±9741.1)
SKESA,Multiple k-mer De Bruijn graph,0.0(±0.0),Genomic,6659581.0(±2881152.41),6124.0(±4725.89),88.66(±7.85),240146.67(±303837.03),4.0(±6.2),117871.0(±180601.9)
SPAdes,Multiple k-mer De Bruijn graph,0.0(±0.0),Genomic,20448859.33(±7028523.19),24923.67(±13809.57),94.15(±5.68),581322.33(±264197.12),191.67(±295.38),57050.67(±54610.55)
Unicycler,Multiple k-mer De Bruijn graph,0.0(±0.0),Genomic,11598783.0(±1617707.48),229.0(±36.54),94.74(±5.29),703922.0(±111322.71),1.0(±1.55),210356.0(±18659.42)


## Global metrics

### Load data

In [14]:
report_glob = glob.glob('../Results/*/report/pipeline_report_tables.json')
reference_pipeline_metrics_df = pd.DataFrame(columns=METRICS_COLUMNS_REFERENCE)

for pipeline_report_file in report_glob:
    report_file_name = pipeline_report_file.split('/')[-1]
    stats_run = pipeline_report_file.split('/')[-3]
    print('Processing {0} data from {1}...'.format(report_file_name, stats_run))
    
    with open(pipeline_report_file) as _fh:
        json_report = json.load(_fh)
        for sample in json_report.keys():
            for reference, data in json_report[sample]['ReferenceTables'].items():
                for row in data:
                    for item in row:
                        reference_pipeline_metrics_df = reference_pipeline_metrics_df.append({'run': stats_run,
                                                                                        'sample': sample,
                                                                                        'assembler': item['assembler'],
                                                                                        'reference': reference,
                                                                                        'contiguity': item['contiguity'],
                                                                                        'breadth_of_coverage': item['breadth_of_coverage'],
                                                                                        'multiplicity': item['multiplicity'],
                                                                                        'validity': item['validity'],
                                                                                        'parsimony': item['parsimony'],
                                                                                        'identity': item['identity'],
                                                                                        'lowest_identity': item['lowest_identity'],
                                                                                        'L90': item['L90'],
                                                                                        'contigs': item['aligned_contigs'],
                                                                                        'NA50': item['NA50'],
                                                                                        'NG50': item['NG50'],
                                                                                        'basepairs': item['aligned_basepairs'],
                                                                                        'Ns': item['Ns'],
                                                                                        'misassemblies': item['misassembled_contigs']},
                                                                                       ignore_index=True)

reference_pipeline_metrics_df['distribution'] = np.where(reference_pipeline_metrics_df['sample'].isin(log_distributed), 'Log', 'Even')
reference_pipeline_metrics_df['type'] = np.where(reference_pipeline_metrics_df['assembler'].isin(genomic_assemblers), 'Genomic', 'Metagenomic')
reference_pipeline_metrics_df['algorythm'] = np.where(reference_pipeline_metrics_df['assembler'].isin(single_kmer), 'Single k-mer De Bruijn graph', 'Multiple k-mer De Bruijn graph')
reference_pipeline_metrics_df[['contigs','basepairs','L90','Ns','NA50','NG50','misassemblies', 'multiplicity','validity','parsimony','identity','lowest_identity']] = reference_pipeline_metrics_df[['contigs','basepairs','L90','Ns','NA50','NG50','misassemblies','multiplicity','validity','parsimony','identity','lowest_identity']].apply(pd.to_numeric)
display(reference_pipeline_metrics_df)

Processing pipeline_report_tables.json data from run3...
Processing pipeline_report_tables.json data from run2...


,run,sample,assembler,reference,contiguity,breadth_of_coverage,multiplicity,validity,parsimony,identity,...,L90,contigs,NA50,NG50,basepairs,Ns,misassemblies,type,algorythm,distribution
0,run3,ENN,BCALM2,Bacillus_subtilis,0.032648,0.976835,1.000279,1.000000,1.000279,1.000000,...,66,116,65138,64934,3951958,0,0,Genomic,Single k-mer De Bruijn graph,Even
1,run3,ENN,GATBMiniaPipeline,Bacillus_subtilis,0.097833,0.983819,1.000000,1.000000,1.000000,0.999966,...,14,29,269695,263366,3980213,0,0,Metagenomic,Multiple k-mer De Bruijn graph,Even
2,run3,ENN,IDBA-UD,Bacillus_subtilis,0.077894,0.982940,1.000048,0.999999,1.000048,0.999558,...,21,38,211169,211169,3976658,0,0,Metagenomic,Multiple k-mer De Bruijn graph,Even
3,run3,ENN,MEGAHIT,Bacillus_subtilis,0.097848,0.985261,1.000217,0.999861,1.000356,0.998712,...,14,28,263587,263587,3986046,0,0,Metagenomic,Multiple k-mer De Bruijn graph,Even
4,run3,ENN,metaSPAdes,Bacillus_subtilis,0.255293,0.985404,1.000000,1.000000,1.000000,0.998464,...,10,20,325068,325068,3986625,0,0,Metagenomic,Multiple k-mer De Bruijn graph,Even
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1435,run2,LHS,MINIA,Staphylococcus_aureus_plasmid3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,Genomic,Single k-mer De Bruijn graph,Log
1436,run2,LHS,SKESA,Staphylococcus_aureus_plasmid3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,Genomic,Multiple k-mer De Bruijn graph,Log
1437,run2,LHS,SPAdes,Staphylococcus_aureus_plasmid3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,Genomic,Multiple k-mer De Bruijn graph,Log
1438,run2,LHS,Unicycler,Staphylococcus_aureus_plasmid3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,Genomic,Multiple k-mer De Bruijn graph,Log


In [15]:
reference_pipeline_metrics_df.dtypes

run                     object
sample                  object
assembler               object
reference               object
contiguity             float64
breadth_of_coverage    float64
multiplicity           float64
validity               float64
parsimony              float64
identity               float64
lowest_identity        float64
L90                      int64
contigs                  int64
NA50                     int64
NG50                     int64
basepairs                int64
Ns                       int64
misassemblies            int64
type                    object
algorythm               object
distribution            object
dtype: object

### Save data

In [16]:
reference_pipeline_metrics_df.to_csv("Tables/Results/Reference metrics.csv")

### Reference statistics per assembler type

In [17]:
ref_stats_per_assembler_type = pd.DataFrame()
for assembly_type in reference_pipeline_metrics_df.type.unique():
    row={'Assembler Type': assembly_type}
    lala=reference_pipeline_metrics_df[reference_pipeline_metrics_df.type == assembly_type].describe()
    for column in lala.columns:
        mean = lala.loc['mean',column]
        std = lala.loc['std',column]
        row[column] = "{}(±{})".format(round(mean, 2), round(std, 2))
    ref_stats_per_assembler_type=ref_stats_per_assembler_type.append(row, ignore_index=True)
ref_stats_per_assembler_type = ref_stats_per_assembler_type.set_index('Assembler Type')
ref_stats_per_assembler_type.to_csv("Tables/Results/Reference metrics per assembler type.csv")

In [18]:
ref_stats_per_assembler_type

,L90,NA50,NG50,Ns,basepairs,breadth_of_coverage,contigs,contiguity,identity,lowest_identity,misassemblies,multiplicity,parsimony,validity
Assembler Type,,,,,,,,,,,,,,
Genomic,32.27(±116.14),44541.82(±91876.27),44301.36(±93249.33),81.46(±801.06),1392591.77(±2023755.37),0.42(±0.47),113.05(±296.71),0.08(±0.2),0.5(±0.5),0.47(±0.49),1.55(±27.23),0.5(±0.5),0.51(±0.51),0.5(±0.5)
Metagenomic,24.19(±51.73),82725.76(±123505.97),81989.47(±122524.9),0.0(±0.0),1830606.01(±2236330.94),0.55(±0.49),45.2(±82.46),0.14(±0.26),0.66(±0.47),0.61(±0.46),0.39(±1.08),0.67(±0.48),0.69(±0.56),0.66(±0.47)


### Reference stats per assembly algorithm

In [19]:
ref_stats_per_assembler_algo = pd.DataFrame()
for assembly_algo in reference_pipeline_metrics_df.algorythm.unique():
    row={'Assembler Algorithm': assembly_algo}
    lala=reference_pipeline_metrics_df[reference_pipeline_metrics_df.algorythm == assembly_algo].describe()
    for column in lala.columns:
        mean = lala.loc['mean',column]
        std = lala.loc['std',column]
        row[column] = "{}(±{})".format(round(mean, 2), round(std, 2))
    ref_stats_per_assembler_algo=ref_stats_per_assembler_algo.append(row, ignore_index=True)
ref_stats_per_assembler_algo = ref_stats_per_assembler_algo.set_index('Assembler Algorithm')
ref_stats_per_assembler_algo.to_csv("Tables/Results/Reference metrics per assembler algorithm.csv")

In [20]:
ref_stats_per_assembler_algo

,L90,NA50,NG50,Ns,basepairs,breadth_of_coverage,contigs,contiguity,identity,lowest_identity,misassemblies,multiplicity,parsimony,validity
Assembler Algorithm,,,,,,,,,,,,,,
Single k-mer De Bruijn graph,59.17(±180.12),8875.39(±15789.06),8406.65(±15616.07),0.0(±0.0),1281528.14(±1944494.79),0.35(±0.44),200.96(±381.74),0.03(±0.13),0.47(±0.5),0.46(±0.5),0.03(±0.25),0.47(±0.5),0.47(±0.5),0.47(±0.5)
Multiple k-mer De Bruijn graph,21.51(±55.64),72550.4(±116246.96),72119.09(±116500.93),61.1(±694.53),1639364.8(±2158330.22),0.5(±0.49),57.14(±174.1),0.12(±0.25),0.59(±0.49),0.54(±0.48),1.34(±23.59),0.59(±0.49),0.6(±0.54),0.59(±0.49)


### Reference stats per assembler

In [21]:
ref_stats_per_assembler = pd.DataFrame()
for assembler in reference_pipeline_metrics_df.assembler.unique():
    row={'Assembler': assembler, 
         'Type':reference_pipeline_metrics_df[reference_pipeline_metrics_df.assembler == assembler].type.unique()[0],
         'Algorithm': reference_pipeline_metrics_df[reference_pipeline_metrics_df.assembler == assembler].algorythm.unique()[0]}
    lala=reference_pipeline_metrics_df[reference_pipeline_metrics_df.assembler == assembler].describe()
    for column in lala.columns:
        mean = lala.loc['mean',column]
        std = lala.loc['std',column]
        row[column] = "{}(±{})".format(round(mean, 2), round(std, 2))
    ref_stats_per_assembler=ref_stats_per_assembler.append(row, ignore_index=True)
ref_stats_per_assembler = ref_stats_per_assembler.set_index('Assembler')
ref_stats_per_assembler.to_csv("Tables/Results/Reference metrics per assembler.csv")

In [22]:
ref_stats_per_assembler

,Algorithm,L90,NA50,NG50,Ns,Type,basepairs,breadth_of_coverage,contigs,contiguity,identity,lowest_identity,misassemblies,multiplicity,parsimony,validity
Assembler,,,,,,,,,,,,,,,,
BCALM2,Single k-mer De Bruijn graph,44.26(±217.6),5388.68(±13681.22),5086.6(±13454.63),0.0(±0.0),Genomic,840447.15(±1650142.0),0.23(±0.38),188.96(±421.77),0.02(±0.12),0.33(±0.47),0.33(±0.47),0.0(±0.0),0.33(±0.47),0.33(±0.47),0.33(±0.47)
GATBMiniaPipeline,Multiple k-mer De Bruijn graph,26.53(±60.32),75511.67(±112427.0),70925.78(±103438.3),0.0(±0.0),Metagenomic,1828000.69(±2238597.0),0.56(±0.49),47.59(±88.73),0.14(±0.27),0.62(±0.49),0.61(±0.48),0.17(±0.53),0.63(±0.49),0.63(±0.49),0.62(±0.49)
IDBA-UD,Multiple k-mer De Bruijn graph,43.74(±74.57),52024.85(±91970.26),51105.3(±91977.8),0.0(±0.0),Metagenomic,1821846.85(±2232054.57),0.54(±0.48),74.9(±120.66),0.09(±0.2),0.66(±0.47),0.57(±0.46),0.64(±1.62),0.67(±0.47),0.74(±0.75),0.65(±0.47)
MEGAHIT,Multiple k-mer De Bruijn graph,12.94(±22.37),105537.11(±143830.26),109204.12(±146844.18),0.0(±0.0),Metagenomic,1837438.15(±2249917.86),0.56(±0.49),31.08(±45.0),0.17(±0.3),0.68(±0.46),0.61(±0.45),0.57(±1.17),0.68(±0.47),0.69(±0.47),0.68(±0.47)
metaSPAdes,Multiple k-mer De Bruijn graph,13.56(±20.69),97829.43(±133394.7),96722.68(±132640.77),0.0(±0.0),Metagenomic,1835138.35(±2248100.93),0.56(±0.49),27.24(±38.26),0.16(±0.27),0.68(±0.47),0.65(±0.46),0.17(±0.5),0.69(±0.47),0.69(±0.48),0.68(±0.47)
MINIA,Single k-mer De Bruijn graph,74.08(±131.58),12362.1(±16990.35),11726.7(±16917.51),0.0(±0.0),Genomic,1722609.12(±2115127.99),0.48(±0.47),212.97(±338.05),0.04(±0.13),0.61(±0.49),0.59(±0.49),0.07(±0.35),0.61(±0.49),0.61(±0.49),0.61(±0.49)
SKESA,Multiple k-mer De Bruijn graph,22.76(±44.46),47300.06(±83407.74),46891.18(±83479.08),0.0(±0.0),Genomic,1436160.07(±1961548.25),0.47(±0.47),95.31(±280.69),0.12(±0.26),0.54(±0.5),0.48(±0.48),0.33(±1.5),0.54(±0.5),0.54(±0.5),0.54(±0.5)
SPAdes,Multiple k-mer De Bruijn graph,25.32(±98.93),83133.97(±116866.68),83465.24(±118060.11),0.0(±0.0),Genomic,1808988.64(±2234202.5),0.52(±0.48),72.15(±260.1),0.12(±0.21),0.7(±0.45),0.64(±0.45),8.28(±66.43),0.72(±0.46),0.73(±0.48),0.7(±0.45)
Unicycler,Multiple k-mer De Bruijn graph,11.68(±21.56),104906.65(±137452.28),101808.62(±136868.91),0.0(±0.0),Genomic,1742439.92(±2215832.77),0.54(±0.49),23.51(±35.61),0.16(±0.26),0.56(±0.5),0.54(±0.49),0.12(±0.53),0.56(±0.5),0.56(±0.5),0.55(±0.5)


### Reference stats per reference replicon

In [23]:
reference_pipeline_metrics_df.reference.unique()

array(['Bacillus_subtilis', 'Enterococcus_faecalis',
       'Escherichia_coli_plasmid', 'Escherichia_coli',
       'Lactobacillus_fermentum', 'Listeria_monocytogenes',
       'Pseudomonas_aeruginosa', 'Salmonella_enterica',
       'Staphylococcus_aureus', 'Staphylococcus_aureus_plasmid1',
       'Staphylococcus_aureus_plasmid2', 'Staphylococcus_aureus_plasmid3'],
      dtype=object)

In [24]:
for reference in reference_pipeline_metrics_df.reference.unique():
    ref_stats_per_reference_per_assembler = pd.DataFrame()
    for assembler in reference_pipeline_metrics_df.assembler.unique():
        row={'Assembler': assembler, 
             'Type':reference_pipeline_metrics_df[reference_pipeline_metrics_df.assembler == assembler].type.unique()[0],
             'Algorithm': reference_pipeline_metrics_df[reference_pipeline_metrics_df.assembler == assembler].algorythm.unique()[0]}
        lala=reference_pipeline_metrics_df[(reference_pipeline_metrics_df.assembler == assembler) & (reference_pipeline_metrics_df.reference == reference)].describe()
        for column in lala.columns:
            mean = lala.loc['mean',column]
            std = lala.loc['std',column]
            row[column] = "{}(±{})".format(round(mean, 2), round(std, 2))
        ref_stats_per_reference_per_assembler=ref_stats_per_reference_per_assembler.append(row, ignore_index=True)
    ref_stats_per_reference_per_assembler = ref_stats_per_reference_per_assembler.set_index('Assembler')
    ref_stats_per_reference_per_assembler.to_csv("Tables/Results/{} - Reference metrics per assembler.csv".format(reference))
    print(reference)
    display(ref_stats_per_reference_per_assembler)

Bacillus_subtilis


,Algorithm,L90,NA50,NG50,Ns,Type,basepairs,breadth_of_coverage,contigs,contiguity,identity,lowest_identity,misassemblies,multiplicity,parsimony,validity
Assembler,,,,,,,,,,,,,,,,
BCALM2,Single k-mer De Bruijn graph,32.5(±51.63),17016.5(±25473.66),16507.83(±25721.82),0.0(±0.0),Genomic,1913231.5(±1818680.2),0.47(±0.45),382.67(±535.65),0.01(±0.01),0.83(±0.39),0.83(±0.39),0.0(±0.0),0.83(±0.39),0.83(±0.39),0.83(±0.39)
GATBMiniaPipeline,Multiple k-mer De Bruijn graph,115.25(±166.23),149601.08(±122100.99),147406.25(±120104.85),0.0(±0.0),Metagenomic,3960376.67(±37546.65),0.98(±0.01),165.42(±215.2),0.08(±0.06),1.0(±0.0),0.99(±0.01),0.58(±0.79),1.0(±0.0),1.0(±0.0),1.0(±0.0)
IDBA-UD,Multiple k-mer De Bruijn graph,92.17(±98.39),111161.67(±94220.24),110292.0(±93518.39),0.0(±0.0),Metagenomic,3968675.5(±15538.85),0.98(±0.0),141.5(±139.84),0.05(±0.03),1.0(±0.0),0.98(±0.01),0.83(±1.11),1.0(±0.0),1.0(±0.0),1.0(±0.0)
MEGAHIT,Multiple k-mer De Bruijn graph,38.67(±51.04),205803.83(±101816.32),206013.67(±101682.93),0.0(±0.0),Metagenomic,3983434.17(±6834.76),0.98(±0.0),65.33(±74.49),0.11(±0.06),1.0(±0.01),0.87(±0.21),0.5(±0.8),1.0(±0.0),1.0(±0.0),1.0(±0.0)
metaSPAdes,Multiple k-mer De Bruijn graph,32.83(±22.3),173649.83(±111545.82),170909.5(±109144.16),0.0(±0.0),Metagenomic,3977137.17(±5153.73),0.98(±0.0),60.0(±37.85),0.13(±0.1),1.0(±0.0),0.98(±0.01),0.67(±0.78),1.0(±0.0),1.0(±0.0),1.0(±0.0)
MINIA,Single k-mer De Bruijn graph,193.33(±216.14),40222.25(±22140.45),39606.08(±22072.87),0.0(±0.0),Genomic,3928355.33(±49914.34),0.97(±0.01),278.67(±242.5),0.02(±0.01),1.0(±0.0),1.0(±0.01),0.67(±0.98),1.0(±0.01),1.0(±0.01),1.0(±0.0)
SKESA,Multiple k-mer De Bruijn graph,9.33(±10.07),117416.17(±124370.34),116361.33(±122979.89),0.0(±0.0),Genomic,1984359.17(±2072596.78),0.49(±0.51),16.33(±17.45),0.05(±0.05),0.5(±0.52),0.5(±0.52),0.0(±0.0),0.5(±0.52),0.5(±0.52),0.5(±0.52)
SPAdes,Multiple k-mer De Bruijn graph,27.17(±23.3),214387.33(±102274.23),222639.33(±109226.61),0.0(±0.0),Genomic,3974502.67(±6095.09),0.98(±0.0),49.33(±34.63),0.13(±0.06),1.0(±0.0),0.99(±0.01),0.33(±0.78),1.0(±0.0),1.0(±0.0),1.0(±0.0)
Unicycler,Multiple k-mer De Bruijn graph,28.83(±52.18),180230.33(±141952.8),162359.67(±156178.13),0.0(±0.0),Genomic,2965228.33(±1564926.8),0.73(±0.39),43.5(±62.9),0.11(±0.08),0.83(±0.39),0.83(±0.39),0.0(±0.0),0.83(±0.39),0.83(±0.39),0.83(±0.39)


Enterococcus_faecalis


,Algorithm,L90,NA50,NG50,Ns,Type,basepairs,breadth_of_coverage,contigs,contiguity,identity,lowest_identity,misassemblies,multiplicity,parsimony,validity
Assembler,,,,,,,,,,,,,,,,
BCALM2,Single k-mer De Bruijn graph,17.5(±40.88),5311.67(±11755.93),5066.58(±11833.0),0.0(±0.0),Genomic,651058.92(±1075258.97),0.23(±0.38),160.42(±303.97),0.01(±0.02),0.33(±0.49),0.33(±0.49),0.0(±0.0),0.33(±0.49),0.33(±0.49),0.33(±0.49)
GATBMiniaPipeline,Multiple k-mer De Bruijn graph,9.17(±10.13),83020.17(±89046.92),84170.5(±90589.23),0.0(±0.0),Metagenomic,1400651.25(±1462979.09),0.49(±0.51),19.25(±20.61),0.08(±0.09),0.5(±0.52),0.49(±0.52),0.33(±0.49),0.5(±0.52),0.5(±0.52),0.5(±0.52)
IDBA-UD,Multiple k-mer De Bruijn graph,32.67(±54.62),41193.67(±54299.03),41736.33(±55119.29),0.0(±0.0),Metagenomic,1393959.67(±1455984.8),0.49(±0.51),56.17(±85.33),0.05(±0.06),0.5(±0.52),0.46(±0.49),0.5(±0.8),0.5(±0.52),0.5(±0.52),0.5(±0.52)
MEGAHIT,Multiple k-mer De Bruijn graph,7.0(±7.48),117668.83(±122901.09),118207.17(±123468.38),0.0(±0.0),Metagenomic,1407900.08(±1470537.2),0.49(±0.52),15.92(±16.69),0.11(±0.12),0.5(±0.52),0.48(±0.5),0.33(±0.49),0.5(±0.52),0.5(±0.52),0.5(±0.52)
metaSPAdes,Multiple k-mer De Bruijn graph,7.5(±9.43),103369.67(±112974.61),103369.67(±112974.61),0.0(±0.0),Metagenomic,1396282.67(±1458429.71),0.49(±0.51),15.17(±18.64),0.08(±0.1),0.5(±0.52),0.49(±0.51),0.17(±0.39),0.5(±0.52),0.5(±0.52),0.5(±0.52)
MINIA,Single k-mer De Bruijn graph,59.25(±63.72),15743.5(±12692.73),13404.83(±14499.77),0.0(±0.0),Genomic,1383381.67(±1432831.1),0.49(±0.5),93.33(±96.61),0.02(±0.02),0.83(±0.39),0.83(±0.39),0.0(±0.0),0.83(±0.39),0.83(±0.39),0.83(±0.39)
SKESA,Multiple k-mer De Bruijn graph,15.83(±17.61),49461.0(±52706.44),47018.5(±52133.86),0.0(±0.0),Genomic,1402020.17(±1458916.42),0.49(±0.51),29.83(±29.71),0.06(±0.06),0.83(±0.39),0.69(±0.4),0.33(±0.49),0.83(±0.39),0.84(±0.39),0.83(±0.39)
SPAdes,Multiple k-mer De Bruijn graph,8.0(±9.63),97227.67(±110100.19),98739.33(±111126.93),0.0(±0.0),Genomic,1412023.67(±1464709.19),0.5(±0.51),21.67(±21.59),0.1(±0.1),0.79(±0.38),0.71(±0.44),1.67(±2.74),0.87(±0.41),0.96(±0.53),0.78(±0.38)
Unicycler,Multiple k-mer De Bruijn graph,4.83(±5.2),141763.17(±155926.65),133592.17(±151708.82),0.0(±0.0),Genomic,1396688.5(±1458835.03),0.49(±0.51),11.5(±12.3),0.13(±0.14),0.5(±0.52),0.5(±0.52),0.0(±0.0),0.5(±0.52),0.5(±0.52),0.5(±0.52)


Escherichia_coli_plasmid


...

,Algorithm,L90,NA50,NG50,Ns,Type,basepairs,breadth_of_coverage,contigs,contiguity,identity,lowest_identity,misassemblies,multiplicity,parsimony,validity
Assembler,,,,,,,,,,,,,,,,
BCALM2,Single k-mer De Bruijn graph,0.0(±0.0),1056.0(±2466.28),1056.0(±2466.28),0.0(±0.0),Genomic,1056.0(±2466.28),0.17(±0.39),0.17(±0.39),0.17(±0.39),0.17(±0.39),0.17(±0.39),0.0(±0.0),0.17(±0.39),0.17(±0.39),0.17(±0.39)
GATBMiniaPipeline,Multiple k-mer De Bruijn graph,0.0(±0.0),3195.92(±3338.67),3195.92(±3338.67),0.0(±0.0),Metagenomic,3165.83(±3306.61),0.5(±0.52),0.5(±0.52),0.5(±0.53),0.5(±0.52),0.5(±0.52),0.0(±0.0),0.5(±0.53),0.5(±0.53),0.5(±0.52)
IDBA-UD,Multiple k-mer De Bruijn graph,0.0(±0.0),1854.83(±2778.99),1854.83(±2778.99),0.0(±0.0),Metagenomic,1849.83(±2773.24),0.29(±0.44),0.33(±0.49),0.29(±0.44),0.33(±0.49),0.33(±0.49),0.0(±0.0),0.33(±0.49),0.33(±0.49),0.33(±0.49)
MEGAHIT,Multiple k-mer De Bruijn graph,0.0(±0.0),3244.5(±3392.54),3244.5(±3392.54),0.0(±0.0),Metagenomic,3168.17(±3309.04),0.5(±0.52),0.5(±0.52),0.5(±0.52),0.49(±0.51),0.49(±0.51),0.0(±0.0),0.5(±0.52),0.51(±0.54),0.49(±0.51)
metaSPAdes,Multiple k-mer De Bruijn graph,0.17(±0.39),2818.17(±3032.84),2818.17(±3032.84),0.0(±0.0),Metagenomic,3146.0(±3286.21),0.5(±0.52),0.67(±0.78),0.44(±0.48),0.5(±0.52),0.5(±0.52),0.0(±0.0),0.5(±0.52),0.5(±0.52),0.5(±0.52)
MINIA,Single k-mer De Bruijn graph,0.0(±0.0),1271.0(±2416.47),1056.0(±2466.28),0.0(±0.0),Genomic,1271.0(±2416.47),0.2(±0.38),0.33(±0.49),0.2(±0.38),0.33(±0.49),0.33(±0.49),0.0(±0.0),0.33(±0.49),0.33(±0.49),0.33(±0.49)
SKESA,Multiple k-mer De Bruijn graph,0.0(±0.0),2287.5(±2829.83),2287.5(±2829.83),0.0(±0.0),Genomic,2819.33(±2972.63),0.44(±0.47),0.83(±1.11),0.36(±0.45),0.5(±0.52),0.49(±0.52),0.0(±0.0),0.5(±0.52),0.5(±0.52),0.5(±0.52)
SPAdes,Multiple k-mer De Bruijn graph,0.0(±0.0),2112.0(±3119.63),2112.0(±3119.63),0.0(±0.0),Genomic,2112.0(±3119.63),0.33(±0.49),0.33(±0.49),0.33(±0.49),0.33(±0.49),0.33(±0.49),0.0(±0.0),0.33(±0.49),0.33(±0.49),0.33(±0.49)
Unicycler,Multiple k-mer De Bruijn graph,0.0(±0.0),3137.5(±3277.37),3137.5(±3277.37),0.0(±0.0),Genomic,3132.5(±3272.01),0.49(±0.52),0.5(±0.52),0.49(±0.52),0.5(±0.52),0.5(±0.52),0.0(±0.0),0.5(±0.52),0.5(±0.52),0.5(±0.52)


Staphylococcus_aureus_plasmid2


,Algorithm,L90,NA50,NG50,Ns,Type,basepairs,breadth_of_coverage,contigs,contiguity,identity,lowest_identity,misassemblies,multiplicity,parsimony,validity
Assembler,,,,,,,,,,,,,,,,
BCALM2,Single k-mer De Bruijn graph,0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),Genomic,0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0)
GATBMiniaPipeline,Multiple k-mer De Bruijn graph,0.0(±0.0),397.0(±927.2),397.0(±927.2),0.0(±0.0),Metagenomic,369.33(±862.58),0.17(±0.39),0.17(±0.39),0.18(±0.42),0.17(±0.39),0.17(±0.39),0.0(±0.0),0.18(±0.42),0.18(±0.42),0.17(±0.39)
IDBA-UD,Multiple k-mer De Bruijn graph,0.0(±0.0),385.83(±901.11),385.83(±901.11),0.0(±0.0),Metagenomic,369.33(±862.58),0.17(±0.39),0.17(±0.39),0.17(±0.41),0.17(±0.39),0.17(±0.39),0.0(±0.0),0.17(±0.41),0.17(±0.41),0.17(±0.39)
MEGAHIT,Multiple k-mer De Bruijn graph,0.0(±0.0),392.83(±917.46),392.83(±917.46),0.0(±0.0),Metagenomic,369.33(±862.58),0.17(±0.39),0.17(±0.39),0.18(±0.41),0.17(±0.39),0.17(±0.39),0.0(±0.0),0.18(±0.41),0.18(±0.41),0.17(±0.39)
metaSPAdes,Multiple k-mer De Bruijn graph,0.0(±0.0),378.5(±883.99),378.5(±883.99),0.0(±0.0),Metagenomic,369.33(±862.58),0.17(±0.39),0.17(±0.39),0.17(±0.4),0.17(±0.39),0.17(±0.39),0.0(±0.0),0.17(±0.4),0.17(±0.4),0.17(±0.39)
MINIA,Single k-mer De Bruijn graph,0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),Genomic,0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0)
SKESA,Multiple k-mer De Bruijn graph,0.0(±0.0),334.17(±780.45),334.17(±780.45),0.0(±0.0),Genomic,334.17(±780.45),0.15(±0.35),0.17(±0.39),0.15(±0.35),0.17(±0.39),0.17(±0.39),0.0(±0.0),0.17(±0.39),0.17(±0.39),0.17(±0.39)
SPAdes,Multiple k-mer De Bruijn graph,0.0(±0.0),326.83(±763.32),326.83(±763.32),0.0(±0.0),Genomic,326.83(±763.32),0.15(±0.34),0.17(±0.39),0.15(±0.34),0.17(±0.39),0.17(±0.39),0.0(±0.0),0.17(±0.39),0.17(±0.39),0.17(±0.39)
Unicycler,Multiple k-mer De Bruijn graph,0.0(±0.0),369.33(±862.58),369.33(±862.58),0.0(±0.0),Genomic,369.33(±862.58),0.17(±0.39),0.17(±0.39),0.17(±0.39),0.17(±0.39),0.17(±0.39),0.0(±0.0),0.17(±0.39),0.17(±0.39),0.17(±0.39)


Staphylococcus_aureus_plasmid3


,Algorithm,L90,NA50,NG50,Ns,Type,basepairs,breadth_of_coverage,contigs,contiguity,identity,lowest_identity,misassemblies,multiplicity,parsimony,validity
Assembler,,,,,,,,,,,,,,,,
BCALM2,Single k-mer De Bruijn graph,0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),Genomic,0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0)
GATBMiniaPipeline,Multiple k-mer De Bruijn graph,0.0(±0.0),525.83(±1228.08),525.83(±1228.08),0.0(±0.0),Metagenomic,498.83(±1165.02),0.17(±0.39),0.17(±0.39),0.18(±0.41),0.17(±0.39),0.17(±0.39),0.0(±0.0),0.18(±0.41),0.18(±0.41),0.17(±0.39)
IDBA-UD,Multiple k-mer De Bruijn graph,0.17(±0.39),281.33(±657.05),281.33(±657.05),0.0(±0.0),Metagenomic,459.5(±1073.16),0.15(±0.36),0.33(±0.78),0.09(±0.22),0.17(±0.39),0.17(±0.39),0.0(±0.0),0.17(±0.4),0.17(±0.4),0.17(±0.39)
MEGAHIT,Multiple k-mer De Bruijn graph,0.0(±0.0),522.33(±1219.91),522.33(±1219.91),0.0(±0.0),Metagenomic,498.83(±1165.02),0.17(±0.39),0.17(±0.39),0.17(±0.41),0.17(±0.39),0.17(±0.39),0.0(±0.0),0.17(±0.41),0.17(±0.41),0.17(±0.39)
metaSPAdes,Multiple k-mer De Bruijn graph,0.0(±0.0),642.83(±1501.34),642.83(±1501.34),0.0(±0.0),Metagenomic,498.83(±1165.02),0.17(±0.39),0.17(±0.39),0.21(±0.5),0.17(±0.39),0.17(±0.39),0.0(±0.0),0.21(±0.5),0.21(±0.5),0.17(±0.39)
MINIA,Single k-mer De Bruijn graph,0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),Genomic,0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0)
SKESA,Multiple k-mer De Bruijn graph,0.0(±0.0),463.67(±1082.89),463.67(±1082.89),0.0(±0.0),Genomic,463.67(±1082.89),0.15(±0.36),0.17(±0.39),0.15(±0.36),0.17(±0.39),0.17(±0.39),0.0(±0.0),0.17(±0.39),0.17(±0.39),0.17(±0.39)
SPAdes,Multiple k-mer De Bruijn graph,0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),Genomic,0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0)
Unicycler,Multiple k-mer De Bruijn graph,0.0(±0.0),498.83(±1165.02),498.83(±1165.02),0.0(±0.0),Genomic,498.83(±1165.02),0.17(±0.39),0.17(±0.39),0.17(±0.39),0.17(±0.39),0.17(±0.39),0.0(±0.0),0.17(±0.39),0.17(±0.39),0.17(±0.39)


### Reference stats per distribution

In [25]:
for distribution in reference_pipeline_metrics_df.distribution.unique():
    ref_stats_per_reference_per_assembler = pd.DataFrame()
    for assembler in reference_pipeline_metrics_df.assembler.unique():
        row={'Assembler': assembler, 
             'Type':reference_pipeline_metrics_df[reference_pipeline_metrics_df.assembler == assembler].type.unique()[0],
             'Algorithm': reference_pipeline_metrics_df[reference_pipeline_metrics_df.assembler == assembler].algorythm.unique()[0]}
        lala=reference_pipeline_metrics_df[(reference_pipeline_metrics_df.assembler == assembler) & (reference_pipeline_metrics_df.distribution == distribution)].describe()
        for column in lala.columns:
            mean = lala.loc['mean',column]
            std = lala.loc['std',column]
            row[column] = "{}(±{})".format(round(mean, 2), round(std, 2))
        ref_stats_per_reference_per_assembler=ref_stats_per_reference_per_assembler.append(row, ignore_index=True)
    ref_stats_per_reference_per_assembler = ref_stats_per_reference_per_assembler.set_index('Assembler')
    ref_stats_per_reference_per_assembler.to_csv("Tables/Results/{} - Reference metrics per assembler.csv".format(distribution))
    print(distribution)
    display(ref_stats_per_reference_per_assembler)

Even


,Algorithm,L90,NA50,NG50,Ns,Type,basepairs,breadth_of_coverage,contigs,contiguity,identity,lowest_identity,misassemblies,multiplicity,parsimony,validity
Assembler,,,,,,,,,,,,,,,,
BCALM2,Single k-mer De Bruijn graph,74.93(±300.64),7322.22(±14758.07),6909.82(±14651.4),0.0(±0.0),Genomic,1290698.65(±1827654.8),0.37(±0.42),342.06(±544.94),0.04(±0.17),0.5(±0.5),0.5(±0.5),0.0(±0.0),0.5(±0.5),0.5(±0.5),0.5(±0.5)
GATBMiniaPipeline,Multiple k-mer De Bruijn graph,29.53(±32.8),109767.99(±118390.68),100966.54(±103290.32),0.0(±0.0),Metagenomic,2524984.11(±2142615.69),0.87(±0.31),57.58(±62.96),0.25(±0.34),0.89(±0.32),0.86(±0.32),0.25(±0.64),0.89(±0.32),0.89(±0.32),0.89(±0.32)
IDBA-UD,Multiple k-mer De Bruijn graph,71.28(±84.72),56036.0(±73769.04),55774.38(±73629.02),0.0(±0.0),Metagenomic,2508176.47(±2128731.97),0.83(±0.34),120.25(±138.91),0.16(±0.27),0.86(±0.35),0.75(±0.39),1.0(±2.14),0.87(±0.35),0.87(±0.35),0.86(±0.35)
MEGAHIT,Multiple k-mer De Bruijn graph,17.47(±17.88),144475.11(±141019.39),151952.72(±144606.65),0.0(±0.0),Metagenomic,2533624.76(±2150186.44),0.88(±0.31),44.06(±44.47),0.3(±0.37),0.89(±0.32),0.79(±0.35),1.08(±1.47),0.89(±0.32),0.9(±0.32),0.89(±0.32)
metaSPAdes,Multiple k-mer De Bruijn graph,19.92(±22.9),134941.08(±125682.7),133991.47(±125319.33),0.0(±0.0),Metagenomic,2529572.39(±2148555.68),0.87(±0.31),38.94(±42.57),0.28(±0.34),0.89(±0.32),0.88(±0.31),0.17(±0.56),0.9(±0.32),0.9(±0.32),0.89(±0.32)
MINIA,Single k-mer De Bruijn graph,82.97(±110.03),18331.39(±18137.17),17666.15(±17998.82),0.0(±0.0),Genomic,2345476.71(±1982106.28),0.72(±0.38),306.47(±391.64),0.08(±0.18),0.81(±0.4),0.79(±0.39),0.0(±0.0),0.81(±0.4),0.81(±0.4),0.81(±0.4)
SKESA,Multiple k-mer De Bruijn graph,38.81(±44.56),84178.14(±88805.36),83571.72(±89085.63),0.0(±0.0),Genomic,2505379.86(±2127723.39),0.85(±0.31),69.67(±78.58),0.23(±0.34),0.89(±0.32),0.77(±0.38),0.33(±0.75),0.89(±0.32),0.89(±0.32),0.88(±0.32)
SPAdes,Multiple k-mer De Bruijn graph,19.72(±20.8),129438.44(±125555.2),129237.0(±125128.06),0.0(±0.0),Genomic,2524523.92(±2144608.71),0.81(±0.37),39.03(±39.85),0.21(±0.26),0.83(±0.38),0.8(±0.38),0.39(±1.01),0.83(±0.38),0.84(±0.38),0.83(±0.38)
Unicycler,Multiple k-mer De Bruijn graph,15.64(±17.3),161424.75(±141031.02),158207.14(±140318.23),0.0(±0.0),Genomic,2521523.86(±2142658.7),0.87(±0.31),32.81(±33.31),0.29(±0.32),0.89(±0.32),0.86(±0.33),0.17(±0.56),0.89(±0.32),0.89(±0.32),0.89(±0.32)


Log


,Algorithm,L90,NA50,NG50,Ns,Type,basepairs,breadth_of_coverage,contigs,contiguity,identity,lowest_identity,misassemblies,multiplicity,parsimony,validity
Assembler,,,,,,,,,,,,,,,,
BCALM2,Single k-mer De Bruijn graph,13.58(±55.44),3455.14(±12312.8),3263.39(±11966.56),0.0(±0.0),Genomic,390195.64(±1316359.92),0.09(±0.27),35.86(±117.43),0.0(±0.01),0.17(±0.38),0.17(±0.37),0.0(±0.0),0.17(±0.38),0.17(±0.38),0.17(±0.38)
GATBMiniaPipeline,Multiple k-mer De Bruijn graph,23.54(±78.96),41255.35(±95190.78),40885.01(±95133.88),0.0(±0.0),Metagenomic,1131017.26(±2125360.46),0.25(±0.43),37.6(±108.12),0.02(±0.06),0.36(±0.48),0.36(±0.48),0.1(±0.38),0.36(±0.48),0.36(±0.48),0.36(±0.48)
IDBA-UD,Multiple k-mer De Bruijn graph,16.21(±49.86),48013.69(±107525.28),46436.22(±107580.39),0.0(±0.0),Metagenomic,1135517.22(±2132472.67),0.25(±0.43),29.56(±76.51),0.02(±0.06),0.46(±0.49),0.4(±0.47),0.28(±0.65),0.47(±0.5),0.61(±0.99),0.44(±0.49)
MEGAHIT,Multiple k-mer De Bruijn graph,8.42(±25.44),66599.11(±136763.09),66455.53(±137159.35),0.0(±0.0),Metagenomic,1141251.53(±2142253.99),0.25(±0.43),18.11(±41.95),0.03(±0.08),0.47(±0.5),0.43(±0.47),0.06(±0.23),0.47(±0.5),0.48(±0.51),0.47(±0.5)
metaSPAdes,Multiple k-mer De Bruijn graph,7.19(±15.98),60717.78(±131337.07),59453.89(±130049.83),0.0(±0.0),Metagenomic,1140704.31(±2141201.79),0.25(±0.43),15.53(±29.28),0.03(±0.07),0.47(±0.5),0.43(±0.48),0.17(±0.44),0.47(±0.5),0.48(±0.51),0.47(±0.5)
MINIA,Single k-mer De Bruijn graph,65.18(±150.35),6392.81(±13423.27),5787.25(±13451.03),0.0(±0.0),Genomic,1099741.54(±2072424.11),0.24(±0.41),119.46(±243.01),0.01(±0.01),0.42(±0.5),0.39(±0.49),0.14(±0.48),0.42(±0.5),0.42(±0.5),0.42(±0.5)
SKESA,Multiple k-mer De Bruijn graph,6.72(±38.38),10421.97(±58025.85),10210.64(±58056.78),0.0(±0.0),Genomic,366940.28(±950611.99),0.1(±0.26),120.94(±388.81),0.01(±0.03),0.19(±0.4),0.19(±0.39),0.33(±1.99),0.19(±0.4),0.19(±0.4),0.19(±0.4)
SPAdes,Multiple k-mer De Bruijn graph,30.92(±138.63),36829.5(±85995.88),37693.47(±90368.89),0.0(±0.0),Genomic,1093453.36(±2101398.69),0.23(±0.41),105.28(±363.93),0.02(±0.05),0.56(±0.48),0.49(±0.47),16.17(±93.6),0.6(±0.52),0.63(±0.55),0.57(±0.48)
Unicycler,Multiple k-mer De Bruijn graph,7.72(±24.6),48388.56(±108092.74),45410.11(±107655.24),0.0(±0.0),Genomic,963355.97(±2016669.48),0.2(±0.39),14.22(±35.63),0.03(±0.06),0.22(±0.42),0.22(±0.41),0.08(±0.5),0.22(±0.42),0.22(±0.42),0.22(±0.42)
